In [ ]:
%pip install stable_baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 42.8 MB/s eta 0:00:00


In [ ]:
%pip install 'shimmy>=0.2.1'

In [ ]:
import gym
from gym import spaces
from stable_baselines3 import PPO, DDPG
import numpy as np

class MassSpringDamperEnv(gym.Env):
    def __init__(self, mass, spring_constant, damping_coefficient):
        self.mass = mass
        self.spring_constant = spring_constant
        self.damping_coefficient = damping_coefficient
        self.dt = 0.01
        self.state = np.array([0.0, 0.0])

        # self.action_space = spaces.Box(low=-10, high=10)
        self.action_space = gym.spaces.Discrete(3, start=-1)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(2,))

    def reset(self):
        temp = np.random.rand(1,2)*np.array([5, 1])
        self.state = temp[0]
        return self.state

    def step(self, action):
        # Apply the action to the system
        force = action * 10
        acceleration = force / self.mass

        # Update the state of the system
        self.state[0] += self.state[1] * self.dt
        self.state[1] += acceleration * self.dt - self.damping_coefficient * self.state[1] * self.dt

        # Calculate the reward
        reward = self.state[0] ** 2

        # Check if the system is done
        done = False

        return self.state, reward, done, {}


In [ ]:
# Create the custom mass-spring-damper environment

# Define the environment
mass = 1.0
spring_constant = 10.0
damping_coefficient = 5.0
env = MassSpringDamperEnv(mass, spring_constant, damping_coefficient)
# Create a PPO agent
# model = PPO("MlpPolicy", env, verbose=1)
model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=100000, log_interval=4)
TIMESTEPS = 10000
for i in range (3):
# model = PPO ("MlpPolicy", env, verbose=1)
  model. learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name="PPO")
  print(i)
# Train the agent
# model.learn(total_timesteps=100)

# Test the trained agent
# obs = env.reset()
# for i in range(100):
#     action, _ = model.predict(obs)
#     obs, reward, done, _ = env.step(action)
#     print(f"Position: {obs[0]}, Velocity: {obs[1]}, Reward: {reward}, i={i}, action: {action}")
#     if done:
#         break

env.close()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 703  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 570          |
|    iterations           | 2            |
|    time_elapsed         | 7            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 8.466013e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.1         |
|    explained_variance   | 1.8e-05      |
|    learning_rate        | 0.0003       |
|    loss                 | 1.25e+08     |
|    n_updates            | 10           |
|    policy_gradient_loss | -1.89e-05    |
|    va

KeyboardInterrupt: ignored

In [ ]:
# Test the trained agent
obs = env.reset()
for i in range(10000):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    print(f"Position: {obs[0]}, Velocity: {obs[1]}, Reward: {reward}, i={i}, action: {action}")
    # if done:
    #     break

env.close()

Streaming output truncated to the last 5000 lines.
Position: 102.76831428698661, Velocity: 2.0054815024518877, Reward: -10561.326421388856, i=5000, action: 2
Position: 102.78836910201113, Velocity: 1.9052074273292934, Reward: -10565.448822651277, i=5001, action: 0
Position: 102.80742117628442, Velocity: 1.8099470559628288, Reward: -10569.365848917934, i=5002, action: 0
Position: 102.82552064684405, Velocity: 1.9194497031646873, Reward: -10573.087696294553, i=5003, action: 2
Position: 102.8447151438757, Velocity: 1.923477218006453, Reward: -10577.035433024936, i=5004, action: 1
Position: 102.86394991605577, Velocity: 2.0273033571061303, Reward: -10580.992192332831, i=5005, action: 2
Position: 102.88422294962683, Velocity: 1.9259381892508238, Reward: -10585.16333194852, i=5006, action: 0
Position: 102.90348233151934, Velocity: 1.8296412797882826, Reward: -10589.126675953314, i=5007, action: 0
Position: 102.92177874431722, Velocity: 1.9381592157988685, Reward: -10592.892539894188, i=5008,

In [ ]:
import gym
import numpy as np
import math
from stable_baselines3 import PPO

# Define the custom Gym environment for the mass-spring-damper system
class MassSpringDamperEnv(gym.Env):
    def __init__(self):
        super(MassSpringDamperEnv, self).__init__()

        # System parameters
        self.m = 1.0  # Mass (kg)
        self.k = 1.0  # Spring constant (N/m)
        self.c = 0.1  # Damping coefficient (N*s/m)

        # Simulation parameters
        self.dt = 0.01  # Time step (s)
        self.max_steps = 1000  # Maximum simulation steps
        self.current_step = 0

        # State and action spaces
        self.action_space = gym.spaces.Box(low=-1.0, high=1.0, shape=(1,))
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(2,))

        # Initial state
        self.state = None

    def reset(self):
        # Reset the environment to an initial state
        self.state = np.array([10.0, 0.0])  # Initial position and velocity
        self.current_step = 0
        return self.state

    def step(self, action):
        # Apply control action and simulate one time step using Euler integration
        force = action[0]
        position, velocity = self.state

        acceleration = (force - self.c * velocity - self.k * position) / self.m
        velocity += acceleration * self.dt
        position += velocity * self.dt

        self.state = np.array([position, velocity])
        self.current_step += 1

        # Calculate the reward (e.g., minimize position error)
        reward = -abs(position)  # Negative position as the reward (minimize position error)

        # Check if the episode is done
        done = self.current_step >= self.max_steps

        return self.state, reward, done, {}

    def render(self, mode='human'):
        pass

    def close(self):
        pass

# Create the custom mass-spring-damper environment
env = MassSpringDamperEnv()

# Create a PPO agent
# model = PPO("MlpPolicy", env, verbose=1)

# # Train the agent
# model.learn(total_timesteps=1000000)

# # Test the trained agent
# obs = env.reset()
# for _ in range(1000):
#     action, _ = model.predict(obs)
#     obs, reward, done, _ = env.step(action)
#     print(f"Position: {obs[0]}, Velocity: {obs[1]}, Reward: {reward}")
#     if done:
#         break

# env.close()

In [ ]:
model_test = PPO.load("MBK_RL_model", env, verbose=1)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.save('MBK_RL_model')

In [ ]:
obs = env.reset()
for i in range(10000):
    action, _ = model_test.predict(obs)
    obs, reward, done, _ = env.step(action)
    if i % 50 == 0:
        print(f"Position: {obs[0]}, Velocity: {obs[1]}, Reward: {reward}, Action: {action}")
    # if done:
        # break

Position: 9.9989, Velocity: -0.11, Reward: -9.9989, Action: [-1.]
Position: 8.596724952288717, Velocity: -5.237847530837125, Reward: -8.596724952288717, Action: [-1.]
Position: 4.971007438385191, Velocity: -8.81995805320879, Reward: -4.971007438385191, Action: [0.6060724]
Position: 0.3608595018665337, Velocity: -9.223399158150176, Reward: -0.3608595018665337, Action: [1.]
Position: -3.8716287230352595, Velocity: -7.403277391927045, Reward: -3.8716287230352595, Action: [1.]
Position: -6.733612520386214, Velocity: -3.903095307697839, Reward: -6.733612520386214, Action: [1.]
Position: -7.608148317735998, Velocity: 0.3588678159672788, Reward: -7.608148317735998, Action: [1.]
Position: -6.383258732809131, Velocity: 4.32712650225246, Reward: -6.383258732809131, Action: [1.]
Position: -3.457825119279758, Velocity: 6.957607031507959, Reward: -3.457825119279758, Action: [-1.]
Position: 0.09597539355331702, Velocity: 6.960110283018015, Reward: -0.09597539355331702, Action: [-1.]
Position: 3.2145